In [ ]:
# visualize the data from training code
# x = input chunk
# and feat2d projected to 3d

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import trimesh
import torch
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
voxel_dims = (1, 1, 1)
data_path = '../inds.pth'
data = torch.load(data_path)
data['coords2d3d'].shape, data['occ3d'].shape

In [ ]:
set([(1,2,3), (4,5,6)]).intersection(set([(4,5,6), (7,8,9)]))

In [ ]:
x = set(list(map(tuple, data['occ3d'].long().tolist())))
y = set(list(map(tuple, data['coords2d3d'].T.long().tolist())))
len(x.intersection(y))

In [ ]:
# 2d coords 
proj_img_size = (40, 30)
path2d = '../2d.pth'
data2d = torch.load(path2d)
i, j = data2d.cpu().numpy()

color_2d = np.zeros(proj_img_size[::-1] + (3,), dtype=int)
# color_2d = np.zeros(proj_img_size + (3,), dtype=int)
color_2d[j, i] = (255, 0, 0)
plt.imshow(color_2d)
plt.axis('off')

In [ ]:
scene = trimesh.scene.scene.Scene() 

print('Draw occupied voxels')
for coord in tqdm(data['occ3d']): 
    i, j, k = coord.tolist()
    # transformation of this voxel
    t = torch.eye(4)
    t[:3, -1] = -torch.Tensor((i, j, k))
    
    voxel_transform = torch.inverse(torch.Tensor(t)).numpy()
    box = trimesh.creation.box(voxel_dims, voxel_transform)
    box.visual.face_colors = np.zeros((12, 4)) + (0, 0, 255, 128)
    box.visual.vertex_colors = np.zeros((8, 4)) + (0, 0, 255, 255)
    scene.add_geometry(box)

print('Draw projected voxels')
for coord in tqdm(data['coords2d3d'].T): 
    i, j, k = coord.tolist()
    # transformation of this voxel
    t = torch.eye(4)
    t[:3, -1] = -torch.Tensor((i, j, k))
    
    voxel_transform = torch.inverse(torch.Tensor(t)).numpy()
    box = trimesh.creation.box(voxel_dims, voxel_transform)
    box.visual.face_colors = np.zeros((12, 4)) + (255, 0, 0, 128)
    box.visual.vertex_colors = np.zeros((8, 4)) + (255, 0, 0, 255)
    scene.add_geometry(box)
       
axes = trimesh.creation.axis(axis_radius=0.1, axis_length=10)
scene.add_geometry(axes)

scene.show()